<a href="https://colab.research.google.com/github/nieyangfei/EGG_Big_Data/blob/main/EEG_Big_Data_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
This is the project of Big Data and Machine Learning. This paper is about Electroencephalogram (EEG)-based emotion recognition. The code is proposed to do these procedure in sequnce.


1.   Data Preparation. Doing this to know what the SEED-IV is like.
2.   SVM classification. This model is easy, at least we can get the passed mark.
3.   CNN classificaiton. This modle is hard, we can do it if time allows.
4.   K Means. We are curious what if we use non labeled network?
4.   Other morden networks. This is hard because we need to find the mordern and special networks. I do not think this is plausible.
## Abbreviations
PSD: Power Spectral Density.
PCA: Principal Component Analysis.
PSD: Power Spectral Density.
DE: Differential Entropy.





# Data Preparation
This will show how the data is.







## File overview by heirachy
1.   **eeg_feature_smooth**. The smooth data of EGG.
Including three sessions, each session has 15 subjects, each subjects has 24 labled variables, each variable has information about 62 electrodes, several time windows, 5 frequency bands.
2.   **eeg_raw_data**. Raw EGG data which has 1000 Hz sampling rate using the ESI NeuroScan System.
3.   **eye_feature_smooth**. The smooth data of eye. The top features are same as eeg_feature_smooth, but the each variable consists of 12 features and several time windows.
4.   **eye_raw_data**. Raw eye data which captured by SMI ETG eye-tracking glasses.
5.   **Channel Order.xlsx**. The EGG electrod name and order in eeg_feature_smooth & eeg_raw_data.
6.   **channel_62_pos**. The 62 EGG channels on the head location. The specific data and head location map does not show.
7.   **ReadMe.txt**. Introduction of the dataset, including copyright, sample gender, label, reference and contact information.
8.   **SEED-IV_stimulation.xlsx**. The video stimulation of the people, which contains video clip of three experiments.
9.   **SEED-IV_subject_info.txt**. The subject gengder, including nine female and six male.

In the Machine learning, this paper will only use eeg_feature_smooth, eye_feature_smooth, ReadMe.txt, Channel Order.xlsx as my input data. Other data in SEED_IV are kinds of middle data that do be used in this paper.

The data is not huge and might currently the author is skeptical about the training result. Fistly, show the EGG and eye data into figures for initial analyze.

As the origin paper says, DE has the best performance. So, this paper tackles with DE features first.

In [1]:
# remove all variables
rm(list = ls())
# optional, free memory
gc()
# Install
if(!require(R.matlab)) install.packages("R.matlab")
if(!require(googledrive)) install.packages("googledrive")
if(!require(devtools)) install.packages("devtools")

# Library / Link
library(R.matlab)
library(googledrive)
library(devtools)

#Initialization
drive_auth()


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657377,35.2,1454471,77.7,1348613,72.1
Vcells,1221086,9.4,8388608,64.0,1975128,15.1


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.methodsS3’, ‘R.oo’, ‘R.utils’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R.matlab v3.7.0 (2022-08-25 21:52:34 UTC) successfully loaded. See ?R.matlab for help.


Attaching package: ‘R.matlab’


The following objects are masked from ‘package:base’:

    getOption, isOpen


Loading required package: usethis

Is it OK to cache OAuth access credentials in the folder ~/.cache/gargle
between R sessions?
1: Yes
2: No


Selection: Yes


Enter a number between 1 and 2, or enter 0 to exit.


Selection: 1


Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/v2/auth?client_id=603366585132-frjlouoa3s2ono25d2l9ukvhlsrlnr7k.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=https%3A%2F%2Fwww.tidyverse.org%2Fgoogle-callback%2F&response_type=code&state=a11d23da78bea13269868641220fff12&access_type=offline&prompt=consent



Enter authorization code: eyJjb2RlIjoiNC8wQVZHelIxQTAtUzR2YXhHbVllYTIzY3R2TXhhaUZDa3VFaE0yNVZsVDVHQ2xuTVYyRVYyZlh1MUpCUS00TkFqNlVrb21mZyIsInN0YXRlIjoiYTExZDIzZGE3OGJlYTEzMjY5ODY4NjQxMjIwZmZmMTIifQ==


In [17]:
## I want to read and show EGG & eye data. To see how it is?


# Read mat file
EGG_sample_file_path_name <- "Project_EGG_Big_Data/SEED_IV/eeg_feature_smooth/1/1_20160518.mat"
print(EGG_sample_file_path_name)
# Find the file in my Google Drive and see it
EGG_sample_file <- drive_get(EGG_sample_file_path_name)
head(EGG_sample_file)
drive_download(EGG_sample_file, path = "1_20160518.mat", overwrite = TRUE)
sample <- readMat("1_20160518.mat")

names(sample)
str(sample, max.level = 1)

[1] "Project_EGG_Big_Data/SEED_IV/eeg_feature_smooth/1/1_20160518.mat"


✔ The input `path` resolved to exactly 1 file.



name          
1 1_20160518.mat
  path                                                              
1 ~/Project_EGG_Big_Data/SEED_IV/eeg_feature_smooth/1/1_20160518.mat
  id                                drive_resource                   
1 1yQ7zovei2PJzekEaBA0X1Omv3WH-ApVZ 1r64a1aynHlpqQyhkNh6w1dYVUVQOmpUv
  <NA>        <NA>       <NA> <NA>                 <NA>                  
1 yangfei nie drive#user TRUE 14809535534033574514 nieyangfeigo@gmail.com
  <NA>                                                                                            
1 https://lh3.googleusercontent.com/a/ACg8ocJKMO1KUsmmRkU5K0lfR9AuJOtYT3vo-EQNpzcsI4D7yQmHsrCZ=s64
  <NA>        <NA>       <NA> <NA>                 <NA>                  
1 yangfei nie drive#user TRUE 14809535534033574514 nieyangfeigo@gmail.com
  <NA>                                                                                            
1 https://lh3.googleusercontent.com/a/ACg8ocJKMO1KUsmmRkU5K0lfR9AuJOtYT3vo-EQNpzcsI4D7yQmHsrCZ=s64
  <NA>             <NA>           <NA>   <NA>   <NA>  <NA>            
1 drive#permission anyoneWithLink anyone writer FALSE drive#permission
  <NA>                 <NA> <NA>                   <NA>  <NA>       
1 14809535534033574514 user nieyangfeigo@gmail.com owner yangfei nie
  <NA>                                                                                            
1 https://lh3.googleusercontent.com/a/ACg8ocJKMO1KUsmmRkU5K0lfR9AuJOtYT3vo-EQNpzcsI4D7yQmHsrCZ=s64
  <NA>  <NA>  <NA>  <NA>  <NA>  <NA>  <NA> <NA> <NA>  <NA> <NA> <NA> <NA> <NA> 
1 FALSE FALSE drive FALSE FALSE FALSE TRUE TRUE FALSE TRUE TRUE TRUE TRUE FALSE
  <NA> <NA> <NA> <NA>  <NA> <NA> <NA> <NA> <NA>  <NA>  <NA> <NA> <NA> <NA> 
1 TRUE TRUE TRUE FALSE TRUE TRUE TRUE TRUE FALSE FALSE TRUE TRUE TRUE FALSE
  <NA> <NA>  <NA>  <NA> <NA> <NA> <NA> <NA> <NA>           <NA>                
1 TRUE FALSE FALSE TRUE TRUE TRUE TRUE TRUE anyoneWithLink 14809535534033574514
  <NA>  <NA> <NA>  <NA>  <NA>  <NA>  <NA>      
1 FALSE TRUE FALSE FALSE FALSE FALSE drive#file
  <NA>                              <NA>           <NA>                    
1 1yQ7zovei2PJzekEaBA0X1Omv3WH-ApVZ 1_20160518.mat application/octet-stream
  <NA>  <NA>  <NA>  <NA>
1 FALSE FALSE FALSE 4   
  <NA>                                                                            
1 https://drive.google.com/uc?id=1yQ7zovei2PJzekEaBA0X1Omv3WH-ApVZ&export=download
  <NA>                                                                               
1 https://drive.google.com/file/d/1yQ7zovei2PJzekEaBA0X1Omv3WH-ApVZ/view?usp=drivesdk
  <NA>                                                                           
1 https://drive-thirdparty.googleusercontent.com/16/type/application/octet-stream
  <NA>  <NA> <NA> <NA>                     <NA>                    
1 FALSE 0    TRUE 2025-10-17T21:46:09.595Z 2025-10-17T21:46:09.595Z
  <NA>                     <NA>                     <NA> <NA> <NA> <NA> <NA> 
1 2018-11-15T15:04:54.000Z 2018-11-15T15:04:54.000Z TRUE TRUE TRUE TRUE FALSE
  <NA> <NA>           <NA> <NA> <NA>                            
1 TRUE 1_20160518.mat mat  mat  7861e59e376101c225e5d2cc72898d89
  <NA>                                    
1 e8d98a4cf5e108a7f5760a3faa903bba12446e8d
  <NA>                                                             <NA>   
1 0c65161c6b064276c855b1774e8a21d0e9ecf1162d1202a7c89dad9c30b19b70 7786107
  <NA>    <NA>                                                <NA>  <NA> 
1 7786107 0B1AUM7oqgWhbcW5XU0UyTEVUbnQ5aEpMR0ROYUorSEpvWmhJPQ FALSE FALSE

File downloaded:

• 1_20160518.mat <id: 1yQ7zovei2PJzekEaBA0X1Omv3WH-ApVZ>

Saved locally as:

• 1_20160518.mat



[1] "de.movingAve1"   "de.LDS1"         "psd.movingAve1"  "psd.LDS1"       
 [5] "de.movingAve2"   "de.LDS2"         "psd.movingAve2"  "psd.LDS2"       
 [9] "de.movingAve3"   "de.LDS3"         "psd.movingAve3"  "psd.LDS3"       
[13] "de.movingAve4"   "de.LDS4"         "psd.movingAve4"  "psd.LDS4"       
[17] "de.movingAve5"   "de.LDS5"         "psd.movingAve5"  "psd.LDS5"       
[21] "de.movingAve6"   "de.LDS6"         "psd.movingAve6"  "psd.LDS6"       
[25] "de.movingAve7"   "de.LDS7"         "psd.movingAve7"  "psd.LDS7"       
[29] "de.movingAve8"   "de.LDS8"         "psd.movingAve8"  "psd.LDS8"       
[33] "de.movingAve9"   "de.LDS9"         "psd.movingAve9"  "psd.LDS9"       
[37] "de.movingAve10"  "de.LDS10"        "psd.movingAve10" "psd.LDS10"      
[41] "de.movingAve11"  "de.LDS11"        "psd.movingAve11" "psd.LDS11"      
[45] "de.movingAve12"  "de.LDS12"        "psd.movingAve12" "psd.LDS12"      
[49] "de.movingAve13"  "de.LDS13"        "psd.movingAve13" "psd.LDS13"      
[53] "de.movingAve14"  "de.LDS14"        "psd.movingAve14" "psd.LDS14"      
[57] "de.movingAve15"  "de.LDS15"        "psd.movingAve15" "psd.LDS15"      
[61] "de.movingAve16"  "de.LDS16"        "psd.movingAve16" "psd.LDS16"      
[65] "de.movingAve17"  "de.LDS17"        "psd.movingAve17" "psd.LDS17"      
[69] "de.movingAve18"  "de.LDS18"        "psd.movingAve18" "psd.LDS18"      
[73] "de.movingAve19"  "de.LDS19"        "psd.movingAve19" "psd.LDS19"      
[77] "de.movingAve20"  "de.LDS20"        "psd.movingAve20" "psd.LDS20"      
[81] "de.movingAve21"  "de.LDS21"        "psd.movingAve21" "psd.LDS21"      
[85] "de.movingAve22"  "de.LDS22"        "psd.movingAve22" "psd.LDS22"      
[89] "de.movingAve23"  "de.LDS23"        "psd.movingAve23" "psd.LDS23"      
[93] "de.movingAve24"  "de.LDS24"        "psd.movingAve24" "psd.LDS24"

List of 96
 $ de.movingAve1  : num [1:62, 1:42, 1:5] 27.8 27.5 27.3 25.5 26 ...
 $ de.LDS1        : num [1:62, 1:42, 1:5] 27.8 27.7 27.7 25.8 26.4 ...
 $ psd.movingAve1 : num [1:62, 1:42, 1:5] 2290388 1856990 1632921 470848 657323 ...
 $ psd.LDS1       : num [1:62, 1:42, 1:5] 3681094 3686479 3803950 704492 1351121 ...
 $ de.movingAve2  : num [1:62, 1:23, 1:5] 28.4 28.9 29.5 24.7 28 ...
 $ de.LDS2        : num [1:62, 1:23, 1:5] 27.5 27.5 27.5 25.5 26.3 ...
 $ psd.movingAve2 : num [1:62, 1:23, 1:5] 3484802 4919637 7490408 268238 2755670 ...
 $ psd.LDS2       : num [1:62, 1:23, 1:5] 3060987 3149514 3358880 569985 1167556 ...
 $ de.movingAve3  : num [1:62, 1:49, 1:5] 25.9 25.5 25.6 24.4 24.8 ...
 $ de.LDS3        : num [1:62, 1:49, 1:5] 26.9 26.8 27 25 25.8 ...
 $ psd.movingAve3 : num [1:62, 1:49, 1:5] 615979 480092 505094 214685 292293 ...
 $ psd.LDS3       : num [1:62, 1:49, 1:5] 6632545 2310361 3243511 421940 907015 ...
 $ de.movingAve4  : num [1:62, 1:32, 1:5] 26.4 27.1 27.9 24.8 26.8 